In [ ]:
print("OK")

OK


In [ ]:
%pwd

'd:\\End-to-End-Medical-chat_bot\\research'

In [ ]:
%pip install langchain-community
#in newer versions need to install Langchain-community
%pip install pypdf


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
#Extract Data from pdf file
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [ ]:
import langchain_community.document_loaders as loaders
print(dir(loaders))
# to check if Directory Loader installed properly in Lnagchain community

['Any', 'TYPE_CHECKING', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__getattr__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_module_lookup', 'base', 'blob_loaders', 'csv_loader', 'dedoc', 'directory', 'helpers', 'html_bs', 'importlib', 'parsers', 'pdf', 'text', 'unstructured']


In [ ]:
%pip install --upgrade langchain-community
!pip list | findstr langchain

In [ ]:
extracted_data=load_pdf(data='D:\End-to-End-Medical-chat_bot\Data')#rememeber to put full path of the Data

In [137]:
#Now We'll perform Chunking operation
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)  #Keep each chunk size to 500 characters #Add a small overlap of 20 characters between chunks so context is preserved
    text_chunks = text_splitter.split_documents(extracted_data)
    unique_chunks = list({
        chunk.page_content: chunk for chunk in text_chunks
    }.values())
    return unique_chunks

In [138]:
text_chunks=text_split(extracted_data)

In [139]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
%pip install -U sentence-transformers 

In [141]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [140]:
embedding=download_hugging_face_embeddings()

In [142]:
query_result = embedding.embed_query("Hi guys Ayush Here")
print("Length",len(query_result)) # to check whether our model is 384 dimensional or not

Length 384


In [ ]:
%pip install -U pybind11

In [ ]:
%pip install numpy==1.26.4


In [ ]:
%pip show numpy

In [ ]:
# query_result

In [151]:
from dotenv import load_dotenv
load_dotenv()
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
PINECONE_API_KEY_1=os.environ.get('PINECONE_API_KEY_1')
HuggingFace_API_KEY=os.environ.get('HuggingFace_API_KEY')

In [134]:
%pip list | findstr pinecone

langchain-pinecone        0.2.5
pinecone                  6.0.2
pinecone-plugin-interface 0.0.7
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import site
site.getsitepackages()


['c:\\Users\\Asus\\anaconda3\\envs\\mchatbot',
 'c:\\Users\\Asus\\anaconda3\\envs\\mchatbot\\lib\\site-packages']

In [153]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "chatbot"

# Create index if it doesn't exist
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(index_name)

# Check if the index already contains vectors
stats = index.describe_index_stats()
vector_count = stats.get("total_vector_count", 0)

if vector_count > 0:
    print(f"Index '{index_name}' already contains {vector_count} vectors (downloads available).")
else:
    print(f"Index '{index_name}' is empty (no downloads yet).")


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '3717df5ea62640452419ea763b1203d1', 'date': 'Thu, 24 Apr 2025 22:36:15 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [152]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY_1)
pc.delete_index("quickstart")

In [156]:
print(pc.list_indexes())

[{
    "name": "chatbot",
    "metric": "cosine",
    "host": "chatbot-1xyhs32.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [155]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY  # by doing this we do not have to create the API key again and again
os.environ["HuggingFace_API_KEY"] = HuggingFace_API_KEY

In [ ]:
%pip install -U langchain-community

In [ ]:
#from langchain_pinecone import PineconeVectorStore
#docsearch = PineconeVectorStore.from_documents(documents=text_chunks,index_name=index_name,embedding=embeddings,)

In [157]:
index = pc.Index(index_name)

# Try a dummy index stats call
stats = index.describe_index_stats()
print(stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [158]:
print(len(text_chunks))


5860


In [ ]:
%pip install -U langchain-pinecone

In [159]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Initialize the Pinecone client (v3 style)
pc = Pinecone(api_key=PINECONE_API_KEY)

# Obtain the actual Pinecone index object (ensure the index already exists)
pinecone_index = pc.Index(index_name)  # e.g., index_name = "quickstart"

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name  # that's all — no pinecone_client or pinecone_index
)
print("Upload complete!")


Upload complete!


In [160]:
#load Existing index
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into our Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [161]:
docsearch

In [163]:
retriever  =docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3}) #is converting our vector store (docsearch) into a retriever so it can be used for semantic search or retrieval-augmented generation (RAG) in a language model pipeline like LangChain.
#"similarity" means it uses vector similarity search (usually cosine similarity) to find the most relevant documents based on your input embedding.
#search_kwargs={"k": 3}:This tells the retriever to return the top 3 most similar chunks (k=3) for each query.
#We are using RAG here inour project


In [164]:
retrieved_docs= retriever.invoke("What is Acne?")

In [165]:
%pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [178]:
import time
import os
from langchain.llms import HuggingFaceHub

# Load a model that supports text generation (e.g., GPT-2)
llm = HuggingFaceHub(
    repo_id="gpt2",  # You can replace this with another model if needed
    model_kwargs={
        "temperature": 0.5, 
        "max_new_tokens": 500       # Limit length of output
    },
    huggingfacehub_api_token=os.environ["HuggingFace_API_KEY"]
)


In [170]:
retrieved_docs

[Document(id='d15f1f8f-8cea-42be-ba56-4f41c01be76e', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'D:\\End-to-End-Medical-chat_bot\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='23ed8496-a101-457e-8027-830f6d26ad00', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'D:\\End-to-End-Medical-chat_bot\\Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ

In [179]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt =(
"You are an assistant for question-answering tasks. "
"Use the following pieces of retrieved context to answer "
"Answer without repeating phrases or sentences."
"If you don't know the answer, say that you don't "
"Use Three sentences maximum and keep the answer concise"
"\n"
"{context}"
)
prompt =ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [180]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
"""def get_answer_with_retry(input_text, retries=5, delay=60):
    for _ in range(retries):
        try:
            # Pass the input as a string (not as a dictionary)
            response = llm.invoke(input_text)
            return response
        except Exception as e:
            print(f"Error encountered: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Retries exhausted. Could not get a valid response.")

# Use the function to get an answer
answer = get_answer_with_retry("What is skin?")
print(answer)"""

In [182]:
def get_answer_with_retry(input_text, retries=5, delay=60):
    for _ in range(retries):
        try:
            response = llm.invoke(input_text)
            return response
        except Exception as e:
            print(f"Error encountered: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Retries exhausted. Could not get a valid response.")

# Use the function to get an answer
answer = get_answer_with_retry("What is Cancer?")
print(answer)


c:\Users\Asus\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


What is Cancer?

The cancer is a complex disease that affects more than one person every day. It is caused by a combination of genetic mutations, genetic mutations, and genetic mutations that affect the way genes work.

When people have cancer, they are more likely to develop certain illnesses, such as heart disease, diabetes, cancer, and other diseases. Many people have a variety of cancers, including cancers of the colon and breast, which can be caused by inherited mutations.

The path of the cancer is different for each person, but it is still possible to develop cancer.

What Causes Cancer?

The main cause of cancer is inherited mutations. These mutations cause a person's immune system to attack certain cells.

A person with a mutation in the genes for the immune system will develop cancer.

Some cancers are caused by a combination of a combination of genetic mutations, genetic mutations, and genetic mutations that affect the way genes work.

Some of the genetic mutations cause can

In [1]:
%pip show numpy 

Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are
        met:
        
            * Redistributions of source code must retain the above copyright
               notice, this list of conditions and the following disclaimer.
        
            * Redistributions in binary form must reproduce the above
               copyright notice, this list of conditions and the following
               disclaimer in the documentation and/or other materials provided
               with the distribution.
        
            * Neither the name of the NumPy Developers nor the names of any
               contributors may be used to endorse or promo